# COMP5318 - Machine Learning and Data Mining - S1 2018

### Group Number: 38
### Tutor Name:Kelvin & Anthony & Harrison 

## Assignment 1 - Due 07 May 2018, 5:00pm

Please refer to [Description File](https://drive.google.com/open?id=1fUFTqcjbSR75tiS48imwjlEVxZVS6U42YElwoVPexwY) for information about the assignment.



#### Group Members (name and student number):


*   Heng Guo 460199841
*   Haoxing Wu 460133111
*   Shen Wang 460255642 





---



## Authenticate and create PyDrive client.

You will be prompted with a link to click on, and give permission to Google Colab to access your Google Drive. If you don't want to give permission to your personal google drive, create a new gmail account, and complete this process using the new account.

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


## Import data
Locate the "Data" folder in your drive. Right click and click "share" to get the ID of the folder. Replace < Data folder id > with the id you got. (id should look like "1j8oG_vCmum965Ghg8LdbSkfj-lfi-AZ0" )

In [0]:
import pandas as pd
import io
import time
import numpy as np

In [3]:
file_list = drive.ListFile({'q': "'1H5y1oS5EmjO9lhNCwKzOg4ciQBS2B14j' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: predicted_labels.csv, id: 1qKwq4fKd8bOw79BpAEO4uaZFaxfD5KJF
title: training_data.csv, id: 1SBp1WGc1lHpRwy0N8qnOvU9q6kSu8Lnh
title: test_data.csv, id: 1TG_-UlxBI03TAensV4GPYbszzwqRlNJ-
title: training_desc.csv, id: 1VxhawDsD_lunMDEsNF4O1Vutudc6HNpT
title: training_labels.csv, id: 1zd8iyfFU3iSHy3ypU8CBnODxoxdPzrnZ


### Pulling data into Google Colab.

In [0]:
training_data_downloaded = drive.CreateFile({'id': '1SBp1WGc1lHpRwy0N8qnOvU9q6kSu8Lnh'})
training_data_downloaded.GetContentFile('training_data.csv')

training_desc_downloaded = drive.CreateFile({'id': '1VxhawDsD_lunMDEsNF4O1Vutudc6HNpT'})
training_desc_downloaded.GetContentFile('training_desc.csv')

training_labels_downloaded = drive.CreateFile({'id': '1zd8iyfFU3iSHy3ypU8CBnODxoxdPzrnZ'})
training_labels_downloaded.GetContentFile('training_labels.csv')

test_data_downloaded = drive.CreateFile({'id': '1TG_-UlxBI03TAensV4GPYbszzwqRlNJ-'})
test_data_downloaded.GetContentFile('test_data.csv') 

test_data_downloaded = drive.CreateFile({'id': '1-eGhlM2cVE4m3e1BNqrsYSyIPHVl5ylk'})
test_data_downloaded.GetContentFile('predicted_labels.csv') 

### Load data files
(example:)

In [0]:
training_file = open('training_data.csv').readlines()
label_file = open('training_labels.csv').readlines()
test_file = open('test_data.csv').readlines()




---


## A. Introduction (max 500 words)


1- What is the aim of the study?


The aim of this study is using a suitable classifier for apps sort. We will classify these apps to different group based on their description. There are huge amounts of apps and their description, we need to choose suitable pre-processing operation to decrease the load of computation. There are several important indicators (accuracy, processing time, precision, recall and f-measure) for evaluation performance. To get higher accuracy results and achieve high performance, the selection and apply of classifier are significant.

2- Why is this study important?

In the process of data processing, it is very important to reduce data and adjust the structure and format of data. Because computers are different in dealing with data with different types of data processing methods and speed, the format of data has great influence on the operation of programs. Obviously, the larger the amount of data is, the longer the processing time will be. Reducing the amount of data helps to improve user experience and save storage space, which means improving economic efficiency.
In the face of today's massive information, the performance of classifiers is an important factor for information retrieval performance. Different classifier algorithms use different methods and processes, and the final results may be different. By completing this task, we need to try different classifiers. This has greatly improved our ability to analyze data and understand the use of classifiers.



## B. Methods (max 1500 words)

### Pre-processing (if any)

*You've been given a few examples of pre-processing during lectues, such as normalisation, removal of outliers and so on. While you are allowed to use external libraries for optimization and linear algebraic calculations, you are NOT allowed to use external libraries for  pre-processing. If you decide to use any form of pre-processing, explain your method clearly here and then write the code for it.*

The format of Apps name and label is str. This kind of format need more time and space to process. Therefore, we give each label a number to represent them. There are 30 labels. Then, we use these numbers to represent feature vectors. Finally, we got the matrix, include all feature vectors belong to this label. it is convenient for computation and processing. We have tryed in different Python Environmental, the results is native Python has best performance compare with pandas and reand_csv.
After that, we tested 2 functions, one is the PCA, another is to filter smaller data.

**PCA**, as a popular procedure to convert huge data into a set of principals, smaller components. First, use each value minus average mean of its column. Then calculate the covariances matrix as well as its eigenvalues and eigenvectors. Then sort the eigenvalue in descending order and get the largest K eigenvalues and its index. Using the data matrix to dot the transposition of the k eigenvectors and get the reduced matrix. However, we found out that it cost too much time to calculate the eigenvalues and eigenvectors for a 13626*13626 matrix as well as the dot multiplication. The test shows that it cost 1328 seconds for the pre-processing module. It did increase the speed of classifier for about 7 times and decrease only a little bit of accuracy for about 0.007. Therefore, with such a long pre-processing time, we choose to not use it for the final processing.

Figure 1. The result of using PCA https://drive.google.com/open?id=1RxVlSeD0pxoCpnujj49wRshKM8AE67o_

**Filter**, In training_data.csv, there are many tf-idf values of words for each app’s name. We consider find a way which based on these tf-idf values to reduce the size of dataset. Firstly, we need know what is tf-idf. TF-IDF (term frequency–inverse document frequency) is a technique which used for data mining and information retrieval. Tf-idf is used to measure the importance of a word in a folder or a file. There is a proportional relationship between the number of occurrences in file and the importance of this word. Search engines always use Tf-idf weighted to evaluate the interdependency between file and information which consumers wants to find. 
Therefore, based on this method, delete a part of columns which the value of tf-idf value are extremely low is a feasible way to reduce size of dataset. Because, the extremely low value means that this word is not important for retrieval. 
First we analyzed the original data, we calculate the average mean for each column, sorted them and plot it in a picture as figure 2. It shows that the mean suddenly increased in a period time between 0 to 0.001, so we tested several times and found out the suitable threshold is 0.0001. So, we delete those columns whose average means are less than 0.0001, there are about 7457 columns which meet the requirement. The result also shows that it only needs 8 more seconds for the filter and increase the speed for about 4 times. Moreover, the accuracy has nearly no decreasing.

Figure 2. Filter https://drive.google.com/open?id=1C8AP6DBtvlykihf4idtWO17Yd3Bu2jmV


In [7]:
app_name = {}
label = []
training_list = [[]]*10
test_data = []
deleteColumn = []

def init():
    start=time.time()
    global training_file,label_file,test_file,app_name,deleteColumn
    extra = len(training_file)%10
    line_num = int(len(training_file)/10)
    line_list = [0]*10
    for i in range(0,10):
        if i < extra:
            line_list[i] = line_num+1
        else:
            line_list[i] = line_num
    for l in label_file:
        l = l.replace('\n','').split(',')
        app_name[l[0]]=l[1]
    for l in app_name.values():
        if l not in label:
            label.append(l)
    label_file = []

    temp = []
    for l in training_file:
        l = l.replace('\n','').split(',')
        l[0] = label.index(app_name[l[0]])
        temp.append(l)
    count = 0
    
    '''
    ##commit PCA
    trainingMatrix = np.array(temp, dtype=np.float)
    training_matrix = np.delete(trainingMatrix,0,1)
    row, column=training_matrix.shape
    k = 450
    for i in range(0,column):
        training_matrix[:,i] = training_matrix[:,i]-np.sum(training_matrix[:,i])/row
    cov_matrix = np.cov(training_matrix.T)
    eigValue,eigVect = np.linalg.eig(cov_matrix)
    eigValIndice = np.argsort(eigValue)
    k_eigValIndice = eigValIndice[-1:-(k+1):-1]   
    k_eigVect = eigVect[:,k_eigValIndice]         
    lowMatrix = training_matrix.dot(k_eigVect)
    temp = np.insert(lowMatrix, 0, values=trainingMatrix[:,0].T, axis=1)
    
    ##commit end
    '''
    ##commit delete
    trainingMatrix = np.array(temp, dtype=np.float)
    training_matrix = np.delete(trainingMatrix,0,1)
    row, column = training_matrix.shape
    for j in range(0,column):
        if (np.sum(training_matrix[:,j])/row)<0.0001:
            deleteColumn.append(j)
    lowDataMatrix = np.delete(training_matrix, deleteColumn, axis = 1)
    temp = np.insert(lowDataMatrix, 0, values=trainingMatrix[:,0].T, axis=1)
    print('delete {} columns'.format(len(deleteColumn)))
    ##commit end
    
    training_file = []
    for i in range(0,10):
        training_list[i] = np.array(temp[count:count+line_list[i]],dtype=np.float)
        count+=line_list[i]
    temp=[]
    app_name=[]
    print('init time:',time.time()-start)
    
init()

delete 7457 columns
init time: 47.631765604019165


### Classifier

*You have to implement the classiﬁer yourself. Same as with pre-processing, you are not allowed to use external libraries for classification. In this section, clearly explain your classifier followed by your code. Your description for pre-processing and classifier should not be more than 1500 words in total.*

For both classifier, we use 10 fold cross validation to calculate the average accuracy.

**Naïve Bayes**

Naïve Bayes is one of the classical machine learning algorithms, and it is also one of the few classification algorithms based on probability theory. Naive Bayes principle is simple and easy to implement. It is mostly used for text categorization, such as spam filtering.
The core of naive Bayes is Bayes rule, and the cornerstone of Bayes rule is conditional probability.

Figure 7 Bayes Rule https://drive.google.com/open?id=1S3RDjL60sbcX1Cbh8S_rbWkTPJfIGFoE

Firstly, we divide the feature vectors into 30 categories according to the label. Because the matrix is sparse matrix, we use 1e-300 represent 0.
Using an dict struct to store the data, key is the label and values are feature vetores and each line represent for one app. Based on Bayes rule, for each label matrix, count how many apps in it and divided by the total number of apps as P(y)=F(y)/N. In each label, the column is one features as Xi, sum the coulumn and then divided by the total number of features as P(xi|y). So far, we have the P(y) and P(x|y) and can predict.
To avoid underflow, we take logarithms of the product of probability.

**Logistic regression**

Firstly, to facilitate the next operation, using a dict struct to store the data, key is the label and values are feature vetores and each line represent for one app. 
Logistic regression is method that when deal with a regression or classification problem, build a cost function, then the optimal model parameters are solved iteratively by the optimization method. Finally, to checkout if our method is useful by testing and verification. Actually, logical regression is a classification method. Usually, use this method to deal with problems which has two kings of output, each one representing one category. Logical regression is always used to solve probability problem.

Figure 3 Logistic Regression https://drive.google.com/open?id=1-l21RhA6FSOcFKk3ot_cZAqnvknTh_IO

Figure 4 The function diagram of sigmoid https://drive.google.com/open?id=1bbYaAWAjOgjabcOqQzQP_4qoVCoP44k1

The figure 4 shows that sigmoid’s output is between (0,1), 0.5 is the intermediate number. Therefore, hϴ(x) is between 0 to 1. If hϴ(x)<0.5 means this data belong to same type. If hϴ>0.5 means this data belong to another type.
Based on this formula, we can estimate the parameter ϴ. The meaning of ϴ is the probability when h ϴ(x)=1, thus, for input x classification and results is:

                            P(y=1|x;0) =h ϴ(x)
                            
                            P(y=0|x;0) = 1- h ϴ(x)

According to these equations, we use maximum likelihood estimation to get the solution of cost function. 


Figure 5 Maximum likelihood https://drive.google.com/open?id=1-l21RhA6FSOcFKk3ot_cZAqnvknTh_IO

To calculate the result, Gradient Descent is an effective method, we use this method to get the best solution.




In [0]:
class NB:
    def __init__(self):
        self.x_prop = {}
        self.y_prop = {}
        self.matrix = np.zeros((30,30))
        
    ##dict:{[label1:[x1,x2,x3...xn],[x1,x2,...]], [label2:....],...}
    def separate(dataset):
        separate = {}
        for i in range(len(dataset)):
            vector = dataset[i]
            label = int(vector[0])
            if label not in separate:
                separate[label] = []
            separate[label].append(np.array(vector[1:]))
        return separate
    
    def calculate(dataset):
##        f = np.array([sum(x) for x in zip(*dataset)])
##        N = np.array(dataset).sum()
##        prob = f/N
        prob = np.array([sum(x) for x in zip(*dataset)])
        prob = np.where(prob==0,1e-300,prob)
        return np.log(prob)
        
    def predict(self,dataset):
        p = {}
        prediction = []
        N = len(dataset)
        c = 0
        for i in range(0,len(dataset)):
            label = dataset[i][0]
            vector = np.array(dataset[i][1:],dtype=np.float)

            for key in list(self.x_prop.keys()):
                p[key] = sum(vector*self.x_prop[key])+self.y_prop[key]
            p_t = {k:v for v,k in p.items()}

            prediction.append(p_t[max(p.values())])
        for i in range(len(dataset)):
            self.matrix[int(dataset[i][0])][int(prediction[i])] += 1
            if prediction[i] == dataset[i][0]:
                c+=1
##        print(prediction)
        acc = c/N
        print('Accuracy ={}'.format(c/N))
        return acc
        
    def train(self):
        start = time.time()
        test = []
        label_feature = {}
        acclist=[]
        global training_list
        
        ##Ten-fold-cross-validation
        for i in range(0,10):
             for j in range(0,10):
                 if j == i:
                     test = training_list[j]
                 else:
                     label_feature = NB.separate(training_list[j])
                     label_list = list(label_feature.keys())
                     for key in label_list:
                         if key in self.x_prop.keys():
                             self.x_prop[key] += np.array(NB.calculate(label_feature[key]))
                         else:
                             self.x_prop[key] = np.array(NB.calculate(label_feature[key]))

                     for key in label_list:
                         if key in self.y_prop.keys():
                             self.y_prop[key] += len(label_feature[key])
                         else:
                             self.y_prop[key] = len(label_feature[key])
             N = sum(a.y_prop.values())
             for key in list(self.y_prop.keys()):
                 self.y_prop[key] = np.log(self.y_prop[key]/N)
             acclist.append(NB.predict(self,test))
             if i < 9:
                 self.x_prop = {}
                 self.y_prop = {}
        end = time.time()
        print('average accuracy =',np.array(acclist).mean())
        print('runing time:',end-start)
        label_feature = NB.separate(training_list[9])
        label_list = list(label_feature.keys())
        for key in label_list:
            if key in self.x_prop.keys():
                self.x_prop[key] += np.array(NB.calculate(label_feature[key]))
            else:
                 self.x_prop[key] = np.array(NB.calculate(label_feature[key]))

        for key in label_list:
            if key in self.y_prop.keys():
                self.y_prop[key] += len(label_feature[key])
            else:self.y_prop[key] = len(label_feature[key])


class LR:
    def __init__(self):
        self.weights = {}
        self.matrix = np.zeros((30,30))
        self.prediction = []
    ##dict:{[label1:[x1,x2,x3...xn],[x1,x2,...]], [label2:....],...}
    def separate(dataset):
        separate = {}
        for i in range(len(dataset)):
            vector = dataset[i]
            label = int(vector[0])
            if label not in separate:
                separate[label] = []
            separate[label].append(np.array(vector[1:]))
        return separate
    
    def sigmoid(x):
        return 1.0 / (1 + np.exp(-x))
        
    def lr(self,dataset,w):
        m,n = np.shape(dataset)
        max_iter = 10
        y = 1
        for j in range(max_iter):
            dataIndex=list(range(m))
            for i in range(m):
                alpha=4/(1+i+j)+0.001
                randIndex=int(np.random.uniform(0,len(dataIndex)))
                h = LR.sigmoid(dataset[randIndex] * w)
                error= y - h
                w = w + alpha*dataset[randIndex]*error
                del(dataIndex[randIndex])
        return w 

    def train(self):
        acclist = []
        global training_list
        start = time.time()
        ##Ten-fold-cross-validation
        for i in range(0,10):
             for j in range(0,10):
                 if j == i:
                     test = training_list[j]
                 else:
                     label_feature = LR.separate(training_list[j])
                     for key in label_feature:
                         if key not in self.weights:
                             w = np.ones((len(training_list[j][0])-1))
                             self.weights[key]=LR.lr(self,label_feature[key],w)
                         else:
                             self.weights[key]=LR.lr(self,label_feature[key],self.weights[key])
             acclist.append(LR.predict(self,test))
             print('accuracy:',acclist[i])
             if i < 9:
                 self.weights = {}
        end = time.time()
        print('average accuracy =',np.array(acclist).mean())
        print('runing time:',end-start)
        label_feature = LR.separate(training_list[9])
        for key in label_feature:
            if key not in self.weights:
                w = np.ones((len(training_list[j][0])-1))
                self.weights[key]=LR.lr(self,label_feature[key],w)
            else:
                self.weights[key]=LR.lr(self,label_feature[key],self.weights[key])
    def predict(self,dataset):
        global label
        prediction = []
        p = {}
        c = 0
        N = len(dataset)
        for i in range(len(dataset)):
            xi = np.array(dataset[i][1:])
            for key in self.weights:
                p[key] = float(xi.dot(self.weights[key]))
            p_t = {k:v for v,k in p.items()}
            prediction.append(p_t[max(p.values())])
        for i in range(len(dataset)):
            self.matrix[int(dataset[i][0])][int(prediction[i])] += 1
            if prediction[i] == dataset[i][0]:
                c+=1
##        print(prediction)
        self.prediction = prediction
        for i in range(len(self.prediction)):
            self.prediction[i] = label[int(self.prediction[i])]
        acc = c/N
##        print('Accuracy ={}'.format(c/N))
        return acc

## C. Experiments and Results (max 500 words)

*We expect you to provide a rigorous performance evaluation. To provide an estimate of the performance (precision, recall, F-measure, etc.) of your classiﬁer in the report, you can perform a 10-fold cross validation on the training set provided and average the metrics for each fold.*

### Accuracy

We have used ten-fold cross validation to test accuracy. This method is an effective way for precision testing. We divided the data set into ten parts, 9 of all dataset are trained and use one part of dataset for testing. Then, each part of dataset to do once testing, others as training data. After that, calculate the mean value of all outcomes. This result represents our estimation of the accuracy of the algorithm. If a higher precise is needed, this process could be executed many times. In our program, we did 10 times 10 times cross validation. The average accuracy is 0.6103277248763641. Runing time is 217.59455585479736s.

In [9]:
if __name__ == '__main__':
##    init()
##    a=NB()
##    a.train()
    b=LR()
    b.train()
    appname = []
    test_data = []
    for l in test_file:
        l = l.replace('\n','').split(',')
        appname.append(l[0])
        l[0] = 0
        test_data.append(l)
    

accuracy: 0.6136250621581303
accuracy: 0.6016907011437096
accuracy: 0.6001989060169071
accuracy: 0.5972153157633019
accuracy: 0.5990049751243781
accuracy: 0.6169154228855721
accuracy: 0.6268656716417911
accuracy: 0.618407960199005
accuracy: 0.6114427860696517
accuracy: 0.6179104477611941
average accuracy = 0.6103277248763641
runing time: 217.59455585479736


### Extensive Analysis



There are 30 categories in the data, so this part is basically using a 30*30 confusion matrix to calculate the precision, recall, f-measure and the accuracy. Based on the definition of these concepts, the matrix uses the column as predictions and the row as golden standards. the equations are as followed:

Precision = right number / the total number of its column

Recall = right number / the total number of its row

F-measure = 2*right number / (the total number of its column+row)

Accuracy = right number/total number

All of precision, recall and f-measure are calculated separately for each class and then the result shows their average mean.


In [10]:
  def getCost(matrix_30_30) :
      precision_total=0
      recall_total=0
      f_meature_total=0
      precision_average=0
      recall_average=0
      f_meature_average=0
      right=0
      total=np.sum(matrix_30_30)

      for i in range(30):
          precision_total+= matrix_30_30[i][i]/(np.sum(matrix_30_30[:,i]))
          recall_total+= matrix_30_30[i][i]/(np.sum(matrix_30_30[i]))
          f_meature_total+= 2*matrix_30_30[i][i]/(np.sum(matrix_30_30[:,i])+np.sum(matrix_30_30[i]))
          right+=matrix_30_30[i][i]
      precision_average = precision_total/30
      recall_average = recall_total/30
      f_meature_average = f_meature_total/30
      accuracy = right/total
      print ('Average precision is : {}\nAverage recall is : {}\nAverage f-meature is : {}\nAccuracy is : {}'.format(precision_average, recall_average, f_meature_average, accuracy))
      ##return (precision_average, recall_average, f_meature_average, accuracy)
      
  b.matrix = pd.DataFrame(b.matrix)
  print(b.matrix)
  getCost(b.matrix.values)


       0      1      2      3      4      5      6      7      8      9   \
0   637.0    4.0    0.0    2.0    0.0    0.0    2.0    0.0    4.0    3.0   
1    66.0  383.0    1.0    1.0    6.0   66.0   11.0    6.0    6.0    4.0   
2    50.0    4.0  148.0    1.0    8.0   21.0    6.0   14.0   31.0    8.0   
3     3.0    0.0    1.0  419.0    0.0    0.0    1.0    1.0    3.0   44.0   
4     6.0    7.0    0.0    3.0  525.0    3.0    6.0   32.0    8.0    4.0   
5     8.0   54.0    2.0    0.0    0.0  426.0    5.0    9.0   36.0    1.0   
6     9.0    6.0    0.0    0.0    2.0    5.0  589.0    0.0    1.0    1.0   
7     2.0    2.0    0.0    3.0    4.0    5.0    1.0  597.0    9.0    0.0   
8    26.0    4.0    1.0    3.0    4.0   75.0    7.0   10.0  262.0    5.0   
9     6.0    0.0    0.0   42.0    1.0    1.0    0.0    6.0    0.0  528.0   
10   23.0    3.0    0.0    1.0    5.0    4.0    3.0   17.0    4.0   43.0   
11    4.0    7.0    0.0    1.0    3.0    6.0    2.0   15.0    2.0    1.0   
12   22.0   

## D. Export Results 

In [157]:
    test_data = np.array(test_data,dtype=np.float)
    test_data = np.delete(test_data,deleteColumn,axis=1)
    b.predict(np.array(test_data,dtype=np.float))
    output = pd.DataFrame({'appname':appname,'label':b.prediction})
    output.to_csv('predicted_labels.csv',header=False,index=False)
    
    uploaded = drive.CreateFile({'title': 'predicted_labels.csv'})
    uploaded.SetContentFile('predicted_labels.csv')
    uploaded.Upload()
    print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1u-j3u0fGVXwGZPUPVUnpeb9fiW2cp_3_


This is important for your grading.


You must save a file named “**predicted_labels.csv**” in the **same data format** as “training_labels.csv”.

You can use PyDrive to save the data file (example is not provided here and you should find out how to do it on your own).

Make sure the predictions (classiﬁcation results for the test set) are in the **same order** as test inputs, i.e. the ﬁrst row of “predicted_labels.csv” corresponds to the ﬁrst row of “test_data.csv” and so on). 

Your score will be based on how accurate your approach is. We will collect “predicted_labels.csv” and compare it to the actual labels to get the accuracy of your approach. For further testing purposes, we may use a diﬀerent test set while grading.


**Insert the url address of your predicted_labels.csv file: https://drive.google.com/file/d/1zd8iyfFU3iSHy3ypU8CBnODxoxdPzrnZ/view?usp=sharing  **



## E. Discussion (max 300 words)

*Provide a meaningful and relevant personal reflection.* 
*Argue around the results from the experiments. Try to explain things like why or why not one classifier performed better than the other in some aspects or why results for a given class were so bad. The discussion is about the results and it should be more technical than philosophical (don't talk about your feelings).*


In the pre-processing step, we have used different wat try to reduce the size of data and change the format of dataset to improve the efficiency of data processing. The original dataset has many data which the format is string. This type of data is easy to identify for humans, but not efficient for computer operation. 

Both PCA and Filter has advantages. PCA spend more time on pre-processing, it takes about 20 minutes. However, it could save time on next step. We found that after the treatment of PCA, the running time is about 100s, shorter than filter. For the total time, filter is faster than PCA. Therefore, for this dataset, filter is a better option.

Contrast Naïve Bayes and logistic regression, the running time of Naïve Bayes is much longer than logistic regression. The running time of Naïve Bates is about 1402s. The running time of logistic regression is about 217s. The reason is in Naïve Bayes method, we use an extremely small number: 1e-300 to represent 0 in sparse matrix. This setting causes computer spends more time on calculation. For logistic regression, this step is not needed. And another factor is the accuracy of Naïve Bayes is lower than logistic regression. The average accuracy is about 61.033% for logistic regression. The average accuracy of Naïve Bayes is 55.735%. 

Figure 8 Resultes of Naive Bayes https://drive.google.com/open?id=1F5rsNPVhbRP2tq3morq_87WPEOuREmx6


## F. Conclusion and future work (max 200 words)

*Provide meaningful conclusions based on your results and suggest meaningful future work.*




We have achieved the main aim of study by using logistic regression classifier. In pre-processing step, we have tried SPA and filter to deal with original dataset. We have also implemented the function with the Naive Bayes method. Finally, we got a 61.033% accuracy. The running time is about 217.595s. 

Through the completion of this project, we learned and practiced a variety of data preprocessing and classifier usage. We have gained more understanding of machine learning principles and implementation. Further work is to try different classifiers, such as Support Vector Machines, and also, there are some other methods we need to learn more for descending dimension, such as Singular Value Decomposition. 

We will try to find some patterns to distinguish which kind of classifier is appropriate for certain type of dataset.There are some factors we need think about: The complexity of the classification function and the size of the training data; The dimension of the input characteristic space; The homogeneity and relationship between input eigenvectors.


## G. References

*You should provide appropriate citations throughout your report and list your references here.*

list of references

1.  Raschka, S. (2014). Implementing a Principal Component Analysis (PCA). Retrieved from http://sebastianraschka.com/Articles/2014_pca_step_by_step.html 
2.  Ray, S. (2017). 6 Easy Steps to Learn Naive Bayes Algorithm (with code in Python). Retrieved from https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/
3.  Li, S. (2017). Building A Logistic Regression in Python, Step by Step. Retrieved from https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
4.  Feller, W. (2008). An introduction to probability theory and its applications (Vol. 2). John Wiley & Sons.
5.  Murphy, K. P. (2006). Naive bayes classifiers. University of British Columbia, 18.

# Sumbitting your assignment

You must share only 1 folder with your tutor.
The folder should have your group's unikeys in its name ( example: Assignment1_abxy6273_edfr3373_yhfr4534 ).

The folder should contain a Colab notebook and the output file.

Suggested structure of the folder:

* Data
 * training_data.csv
 * training_desc.csv
 * training_labels.csv
 * test_data.csv
* Assignment1.ipynb
* predicted_labels.csv

Ask your tutor for his email address and share the properly named folder with him before 5:00pm on May 7th 2018. 
Every late day will cost you 20 marks.

